In [ ]:
#to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt
import pylab as pl
import seaborn as sns

#for the yeo-johnson transform
import scipy.stats as stats

#to display all the columns
pd.pandas.set_option('display.max_columns', None)

In [2]:
#hyperparameters
DATASET='test'
# DATASET='train'

HEATER = 'H160'
MAX_HEAD_PRESSURE = 400
MIN_HEAD_TEMPERATURE = 50
MIN_WEIGHT = 1400

In [3]:
# load the data
df = pd.read_csv('./data/' + DATASET + '/' +HEATER + '.csv', parse_dates=['utc_time'])
df.columns = ['time','batch','phase', 'heat_time', 'weight', 'head_temp', 'head_pressure', 'vac_temp']
# df = df[df['phase'].str.contains('DRYING|HEATING|FEED|FEEDING') == True]

In [4]:
df.sample(40)

,time,batch,phase,heat_time,weight,head_temp,head_pressure,vac_temp
18754,2022-04-16 11:09:18+00:00,157200.0,DRYING,146.3,3254.8,53.00,276.5,24.017
1962,2022-04-04 19:17:18+00:00,157168.0,DRYING,180.0,2913.4,76.00,362.5,17.231
19518,2022-04-16 23:53:18+00:00,157202.0,FEEDING,85.8,2591.0,46.00,251.4,27.997
22956,2022-04-19 09:11:18+00:00,157210.0,FEEDING,98.6,2313.0,59.00,311.3,29.556
1900,2022-04-04 18:15:18+00:00,157168.0,DRYING,180.0,2935.6,76.00,398.3,17.645
9555,2022-04-10 01:50:18+00:00,157184.0,FEEDING,131.9,788.0,48.00,698.5,17.801
14402,2022-04-13 10:37:18+00:00,157192.0,DISCHRG,169.0,227.7,25.33,703.8,29.092
35959,2022-04-28 09:54:18+00:00,157243.0,FEEDING,180.2,2923.4,63.00,541.2,30.041
12457,2022-04-12 02:12:18+00:00,157189.0,FEEDING,38.9,1072.4,44.00,211.0,20.940
17680,2022-04-15 17:15:18+00:00,157199.0,PREPARE,180.0,191.6,18.00,725.0,18.805


In [5]:
from scipy.signal import argrelextrema
# find local maximums
n = 30 # number of points to be checked before and after

df['max_temp'] = 0
df['max_temp'] = df.iloc[argrelextrema(df.head_temp.values, np.greater_equal, order=n)]['head_temp']

In [6]:
df['ready'] = 0
DELTA_TIME = 2  # time after max to be considered ready
DELTA_TEMP = 1 # temperature difference to be considered ready
df['ready'] = df.ready.astype(int)

for i in range(1, len(df)-1):
    if df.max_temp.values[i] > 0 and df.head_temp.values[i] > MIN_HEAD_TEMPERATURE:
        local_max = df.max_temp.values[i]
        current = df.batch.values[i]
        local_max_index = i
        while df['batch'].values[i] == current and i < len(df)-1:
            if local_max - df.head_temp.values[i] > DELTA_TEMP  and i > local_max_index + DELTA_TIME:
                df.ready.values[i] = 1
            i += 1

In [7]:
df.ready.value_counts()

0    21603
1    21586
Name: ready, dtype: int64